# Tutorial 9 - Changing the mesh

In [Tutorial 8](./tutorial-8-solver-options.ipynb) we saw how to change the solver options. In this tutorial we will change the mesh used in the simulation, and show how to investigate the influence of the mesh on the solution.

All models in PyBaMM have a default number of mesh points used in a simulation. However, depending on things like the operating conditions you are simulating or the parameters you are using, you may find you need to increase the number points in the mesh to obtain an accurate solution. On the other hand, you may find that you are able to decrease the number of mesh points and still obtain a solution with an acceptable degree of accuracy but in a shorter amount of computational time. 

It is always good practice to conduct a mesh refinement study, where you simulate the same problem with a finer mesh and compare the results. Here will show how to do this graphically, but in practice you may wish to do a more detailed calculation of the relative error.

In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


## Changing the number of points in the mesh

First we load a model

In [2]:
model = pybamm.lithium_ion.SPMe()

We can then look at the default number of points, which are stored as a dictionary whose keys are the variables for each domain

In [3]:
model.default_var_pts

{'x_n': 20,
 'x_s': 20,
 'x_p': 20,
 'r_n': 20,
 'r_p': 20,
 'y': 10,
 'z': 10,
 'R_n': 30,
 'R_p': 30}

To run a simulation with a different number of points we can define our own dictionary 

In [4]:
# create our dictionary 
var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 10,  # separator 
    "x_p": 10,  # positive electrode
    "r_n": 10,  # negative particle
    "r_p": 10,  # positive particle
}

We then create and solve a simulation, passing the dictionary of points as a keyword argument

In [5]:
sim = pybamm.Simulation(model, var_pts=var_pts)
sim.solve([0, 3600])

and plot the solution in the usual way

In [6]:
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Conducting a mesh refinement study

In order to investigate the influence of the mesh on the solution we must solve the model multiple times, increasing the mesh resolution as we go. We first create a list of the number of points per domain we would like to use

In [7]:
npts = [4, 8, 16, 32, 64]

and now we can loop over the list, creating and solving simulations as we go. The solutions are stored in the list `solutions`

In [8]:
# choose model and parameters
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues("Ecker2015")

# choose solver 
solver = pybamm.CasadiSolver(mode="fast")

# loop over number of mesh points
solutions = []
for N in npts:
    var_pts = {
        "x_n": N,  # negative electrode
        "x_s": N,  # separator 
        "x_p": N,  # positive electrode
        "r_n": N,  # negative particle
        "r_p": N,  # positive particle
    }    
    sim = pybamm.Simulation(
        model, solver=solver, parameter_values=parameter_values, var_pts=var_pts
    )
    sim.solve([0, 3600])
    solutions.append(sim.solution)

We can now pass our list of solutions to the dynamic plot method, allowing use to see the influence of the mesh on the computed voltage. We pass our list of points using the `labels` keyword so that the plots are labeled with the number of points used in the simulation

In [9]:
pybamm.dynamic_plot(solutions, ["Voltage [V]"], time_unit="seconds", labels=npts)    

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=36.0), Output()), _dom_classes=…

In the [next tutorial](./tutorial-10-creating-a-model.ipynb) we show how to create a basic model from scratch in PyBaMM.

## References

The relevant papers for this notebook are:

In [10]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Madeleine Ecker, Stefan Käbitz, Izaro Laresgoiti, and Dirk Uwe Sauer. Parameterization of a Physico-Chemical Model of a Lithium-Ion Battery: II. Model Validation. Journal of The Electrochemical Society, 162(9):A1849–A1857, 2015. doi:10.1149/2.0541509jes.
[4] Madeleine Ecker, Thi Kim Dung Tran, Philipp Dechent, Stefan Käbitz, Alexander Warnecke, and Dirk Uwe Sauer. Parameterization of a Physico-Chemical Model of a Lithium-Ion Battery: I. Determination of Parameters. Journal of the Electrochemical Society, 162(9):A1